In [49]:
# load in libraries
import os
import json
import psycopg2
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas import json_normalize 

In [50]:
# load in env variables
db_host = os.environ['HEROKU_DB_HOST']
db_name = os.environ['HEROKU_DB']
db_user = os.environ['HEROKU_DB_USER']
db_password = os.environ['HEROKU_DB_PASSWORD']

# connect to database
conn = psycopg2.connect(database=db_name, user=db_user, password=db_password, host=db_host, port="5432")
cur = conn.cursor()

In [51]:
# import model
with open('../models/nfl_predictor_rf.pkl', 'rb') as f:
    model = pickle.load(f)

In [52]:
# import gamedata csv
games_dataframe_data = {}
seasons = [2017,2018,2019,2020]
# loop through directory 
for season in seasons:
    # append regular saeason data
    df_reg_season = pd.read_csv('./data/custom_games_by_season/{0}_data.csv'.format(season)) 
    games_dataframe_data[season] = df_reg_season
    

In [53]:
# let model loose to make predictions on all games for each season 
selected_features = ['AwayAverage','AwayFirstDowns', 'AwayTime', 'AwayThirdDowns', 'HomeAverage','HomeFirstDowns', 'HomeTime', 'HomeThirdDowns']

In [54]:
for season, data in games_dataframe_data.items():
    # get values we want
    values = data[selected_features].values
    
    # make predictions
    predictions = model.predict(values)
    
    for index, row in data.iterrows():
        data.at[index, 'PredictHomeTeamWin'] = predictions[index]
        
    # save updated data in csv
    data.to_csv('./data/custom_games_by_season/{0}_data.csv'.format(season), header=True,  encoding='utf-8', index=False) 

    games_dataframe_data[season] = data
    
    

In [55]:
# insert 2020 team custom data into database
df_reg_season = pd.read_csv('./data/custom_season_team_data/2020_data.csv') 

data = df_reg_season.to_json(orient="index")

# create the SQL string
sql_string = 'INSERT INTO team_stats(year, stats) VALUES (%s, %s)'
cur.execute(sql_string, (2020,json.dumps(data)))
conn.commit()


In [56]:
# insert each dataframe into database
for season in seasons:
    data = games_dataframe_data[season].to_json(orient="index")
    
    # create the SQL string
    sql_string = 'INSERT INTO games_data(year, games) VALUES (%s, %s)'
    cur.execute(sql_string, (season,json.dumps(data)))
    conn.commit()

conn.close()